# Library imports

In [1]:
from scipy.stats import percentileofscore as score
from scipy import stats
import pandas as pd
import numpy as np 
import xlsxwriter 
import requests
import math

# Importing Our List of Stocks and getting API Token


In [2]:
pd.set_option('display.max_rows', None)
stocks = pd.read_csv('sp_500_stocks.csv')
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

# Executing a batch API Call & Building our Dataframe

In this section, we will filter for stocks with the lowest percentiles on the following metrics:

    1. Price-to-earnings ratio (Equivalent in portuguese: P/L - Preço sobre lucro)
    2. Price-to-book ratio (Equivalent in portuguese: P/VP - Preço sobre Valor Patrimonial)
    3. Price-to-sales ratio (Equivalent in portuguese: P/S - Preço sobre vendas
    4. Enterprise Value divided by earnings before interest, taxes, depreciation, and amortization (EV/EBITDA)
    5. Enterprise Value divided by gross profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data.

In [3]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol} &types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

print(data['AAPL']['advanced-stats']['priceToBook'])


# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio 
pb_ratio = data['AAPL']['advanced-stats']['priceToBook']

# Price-to-sales ratio 
ps_ratio = data['AAPL']['advanced-stats']['priceToSales']

# Enterprise value divided by earnings before interest, taxes, depreciation, and amortization (EV/EBITDA)
enterprise_value = data['AAPL']['advanced-stats']['enterpriseValue']
ebitda = data['AAPL']['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# Enterprise value divided by Gross Profit (EV/GP)
gross_profit = data['AAPL']['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit


45.14


In [4]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [5]:
def chunks(lst,n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i : i + n]

symbol_groups = list(chunks(stocks['Ticker'],100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    
    symbol_strings.append(','.join(symbol_groups[i]))
    #print (symbol_strings[i])
    
    
for symbol_string in symbol_strings[:6]:      
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string} &types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    
    for symbol in symbol_string.split(','):
        
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']

        # Enterprise value divided by Gross Profit (EV/GP)
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except:
            ev_to_gross_profit = np.NaN
        
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',            
                'N/A'
        ],
        index = rv_columns), ignore_index = True)
        
rv_dataframe
     

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAP,150.250,N/A,17.87,N/A,3.23,N/A,0.797,N/A,9.666644,N/A,2.064629,N/A,N/A
1,AAPL,145.780,N/A,24.17,N/A,46.93,N/A,5.93,N/A,17.896218,N/A,13.914176,N/A,N/A
2,ABBV,171.320,N/A,22.7,N/A,18.63,N/A,5.28,N/A,13.032599,N/A,8.555722,N/A,N/A
3,ABC,169.830,N/A,20.96,N/A,-168.2,N/A,0.1476,N/A,10.961189,N/A,4.451817,N/A,N/A
4,ABMD,396.410,N/A,68.36,N/A,11.34,N/A,16.5,N/A,58.877667,N/A,19.464878,N/A,N/A
5,ABT,109.358,N/A,24.57,N/A,5.37,N/A,4.21,N/A,16.425096,N/A,7.362982,N/A,N/A
6,ACN,292.780,N/A,27.68,N/A,8.99,N/A,3.3,N/A,16.157459,N/A,9.386755,N/A,N/A
7,ADBE,340.760,N/A,33.5,N/A,10.83,N/A,9.28,N/A,25.105935,N/A,10.340178,N/A,N/A
8,ADI,174.290,N/A,33.9,N/A,2.41,N/A,7.37,N/A,20.571909,N/A,12.189233,N/A,N/A
9,ADM,96.165,N/A,13.38,N/A,2.21,N/A,0.5351,N/A,9.589848,N/A,7.546630,N/A,N/A


# Dealing with Missing Data in Our DataFrame
Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

You can use pandas' isnull method to identify missing data.

In [6]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)].index

Int64Index([ 26,  38,  69, 115, 118, 132, 161, 182, 186, 189, 200, 249, 320,
            321, 322, 343, 355, 437, 447, 488],
           dtype='int64')

Dealing with missing data is an important topic in data science.

There are 2 main approaches:

1. Drop missing data from the data set (pandas's dropna method is useful here)
2. Replace missing data with a new value (pandas' fillna method is useful here)

In [7]:
rv_dataframe_filled = rv_dataframe
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe_filled[column].fillna(rv_dataframe[column].mean(), inplace = True)
    

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [8]:
rv_dataframe_filled[rv_dataframe_filled.isnull().any(axis=1)]
rv_dataframe_filled


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAP,150.250,N/A,17.870000,N/A,3.230000,N/A,0.797000,N/A,9.666644,N/A,2.064629,N/A,N/A
1,AAPL,145.780,N/A,24.170000,N/A,46.930000,N/A,5.930000,N/A,17.896218,N/A,13.914176,N/A,N/A
2,ABBV,171.320,N/A,22.700000,N/A,18.630000,N/A,5.280000,N/A,13.032599,N/A,8.555722,N/A,N/A
3,ABC,169.830,N/A,20.960000,N/A,-168.200000,N/A,0.147600,N/A,10.961189,N/A,4.451817,N/A,N/A
4,ABMD,396.410,N/A,68.360000,N/A,11.340000,N/A,16.500000,N/A,58.877667,N/A,19.464878,N/A,N/A
5,ABT,109.358,N/A,24.570000,N/A,5.370000,N/A,4.210000,N/A,16.425096,N/A,7.362982,N/A,N/A
6,ACN,292.780,N/A,27.680000,N/A,8.990000,N/A,3.300000,N/A,16.157459,N/A,9.386755,N/A,N/A
7,ADBE,340.760,N/A,33.500000,N/A,10.830000,N/A,9.280000,N/A,25.105935,N/A,10.340178,N/A,N/A
8,ADI,174.290,N/A,33.900000,N/A,2.410000,N/A,7.370000,N/A,20.571909,N/A,12.189233,N/A,N/A
9,ADM,96.165,N/A,13.380000,N/A,2.210000,N/A,0.535100,N/A,9.589848,N/A,7.546630,N/A,N/A


# Dealing with Price-to-Earnings Ratio < 0

An negative PeRatio indicates that the company has a negative profit, it means, a loss. 
In this approach, I'll remove companies that had a loss.

In [18]:
for row in rv_dataframe_filled.index:
    if (rv_dataframe_filled.loc[row,'Price-to-Earnings Ratio'] < 0):
        rv_dataframe_filled.drop(row)
        print(rv_dataframe_filled.loc[row,'Ticker'])
rv_dataframe_filled

BA
LNC
CAH
ALL
CCL
UAL
CINF
XRX
FTI
CXO
PRGO
BKR
GE
RCL
NCLH
VTR
IFF
NBL
BIO
SLG
BAX
AES
WYNN
TTWO
STE
ILMN
TWTR


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,27.700,7,6.480000,11.044177,0.745000,8.032129,0.165100,0.803213,4.599538,7.831325,1.137719,5.823293,7
1,GM,38.951,5,6.820000,11.84739,0.864000,9.036145,0.383000,4.417671,1.978254,1.807229,1.610315,9.236948,7
2,AIG,63.991,3,3.700000,5.62249,1.200000,13.554217,0.808200,12.248996,2.196776,2.008032,0.785324,2.610442,7
3,C,46.040,4,6.200000,10.240964,0.504000,7.429719,0.964200,14.859438,3.754686,4.618474,1.020876,4.016064,8
4,BA,183.570,1,-12.920000,3.413655,-6.180000,6.024096,1.790000,31.425703,-28.203796,0.60241,-172.958889,0.200803,8
5,COF,94.860,2,4.850000,7.228916,0.715500,7.831325,1.031500,16.666667,3.197437,3.012048,1.077321,5.02008,8
6,UNM,40.950,5,6.960000,12.248996,0.972800,10.441767,0.680500,9.839357,3.900061,5.421687,0.655573,1.405622,8
7,SYF,35.400,6,5.710000,9.236948,1.300000,15.060241,0.885300,13.453815,3.414872,3.614458,1.038622,4.417671,9
8,F,13.950,14,6.250000,10.441767,1.240000,14.257028,0.343000,3.212851,3.956095,5.823293,1.758730,10.040161,9
9,PVH,76.810,3,5.880000,9.638554,0.954100,10.040161,0.574000,7.028112,5.585405,10.843373,1.237095,6.827309,9


# Calculating Value Percentiles
Metrics:

 1. price to earnings ratio
 2. price to book ratio
 3. price to sales ratio
 4. EV/EBITDA
 5. EV/GP

In [10]:
from scipy.stats import percentileofscore as score

metrics = {
    'Price-to-Earnings Ratio': 'PE Percentile',
    'Price-to-Book Ratio' : 'PB Percentile',
    'Price-to-Sales Ratio': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP' : 'EV/GP Percentile',
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAP,150.250,N/A,17.870000,38.654618,3.230000,54.718876,0.797000,11.84739,9.666644,29.317269,2.064629,11.84739,N/A
1,AAPL,145.780,N/A,24.170000,54.819277,46.930000,98.192771,5.930000,83.534137,17.896218,70.682731,13.914176,90.562249,N/A
2,ABBV,171.320,N/A,22.700000,51.004016,18.630000,94.578313,5.280000,79.919679,13.032599,43.574297,8.555722,66.86747,N/A
3,ABC,169.830,N/A,20.960000,47.389558,-168.200000,0.803213,0.147600,0.60241,10.961189,32.931727,4.451817,32.329317,N/A
4,ABMD,396.410,N/A,68.360000,94.176707,11.340000,89.558233,16.500000,99.196787,58.877667,98.995984,19.464878,97.590361,N/A
5,ABT,109.358,N/A,24.570000,58.232932,5.370000,72.289157,4.210000,73.192771,16.425096,66.064257,7.362982,59.036145,N/A
6,ACN,292.780,N/A,27.680000,65.261044,8.990000,84.337349,3.300000,58.634538,16.157459,64.658635,9.386755,72.289157,N/A
7,ADBE,340.760,N/A,33.500000,75.702811,10.830000,88.75502,9.280000,92.068273,25.105935,89.35743,10.340178,78.714859,N/A
8,ADI,174.290,N/A,33.900000,77.309237,2.410000,40.763052,7.370000,87.550201,20.571909,80.522088,12.189233,84.939759,N/A
9,ADM,96.165,N/A,13.380000,27.610442,2.210000,35.441767,0.535100,6.626506,9.589848,28.915663,7.546630,60.64257,N/A


# Calculating RV Score

We'll now calculate our RV Score (Robust Value), which is the value score that we'll use to filter for stocks in this invest strategy. 

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section. 

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module. 

In [11]:
from statistics import mean

for row in rv_dataframe.index:
    pe_ratio = rv_dataframe.loc[row, 'PE Percentile']
    pb_ratio = rv_dataframe.loc[row, 'PB Percentile']
    ps_ratio = rv_dataframe.loc[row, 'PS Percentile']
    ev_to_ebitda = rv_dataframe.loc[row, 'EV/EBITDA Percentile']
    ev_to_gp = rv_dataframe.loc[row, 'EV/GP Percentile']
    rv_dataframe.loc[row, 'RV Score'] = round(mean([pe_ratio, pb_ratio, ps_ratio, ev_to_ebitda, ev_to_gp]))
    
rv_dataframe[['Ticker', 'PE Percentile', 'PB Percentile', 'PS Percentile', 'EV/EBITDA Percentile', 'EV/GP Percentile', 'RV Score']]

,Ticker,PE Percentile,PB Percentile,PS Percentile,EV/EBITDA Percentile,EV/GP Percentile,RV Score
0,AAP,38.654618,54.718876,11.84739,29.317269,11.84739,29
1,AAPL,54.819277,98.192771,83.534137,70.682731,90.562249,80
2,ABBV,51.004016,94.578313,79.919679,43.574297,66.86747,67
3,ABC,47.389558,0.803213,0.60241,32.931727,32.329317,23
4,ABMD,94.176707,89.558233,99.196787,98.995984,97.590361,96
5,ABT,58.232932,72.289157,73.192771,66.064257,59.036145,66
6,ACN,65.261044,84.337349,58.634538,64.658635,72.289157,69
7,ADBE,75.702811,88.75502,92.068273,89.35743,78.714859,85
8,ADI,77.309237,40.763052,87.550201,80.522088,84.939759,74
9,ADM,27.610442,35.441767,6.626506,28.915663,60.64257,32


# Selecting the 10th best value stocks

In [12]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe.head(10)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,27.700,N/A,6.48,11.044177,0.7450,8.032129,0.1651,0.803213,4.599538,7.831325,1.137719,5.823293,7
1,GM,38.951,N/A,6.82,11.84739,0.8640,9.036145,0.3830,4.417671,1.978254,1.807229,1.610315,9.236948,7
2,AIG,63.991,N/A,3.70,5.62249,1.2000,13.554217,0.8082,12.248996,2.196776,2.008032,0.785324,2.610442,7
3,C,46.040,N/A,6.20,10.240964,0.5040,7.429719,0.9642,14.859438,3.754686,4.618474,1.020876,4.016064,8
4,BA,183.570,N/A,-12.92,3.413655,-6.1800,6.024096,1.7900,31.425703,-28.203796,0.60241,-172.958889,0.200803,8
5,COF,94.860,N/A,4.85,7.228916,0.7155,7.831325,1.0315,16.666667,3.197437,3.012048,1.077321,5.02008,8
6,UNM,40.950,N/A,6.96,12.248996,0.9728,10.441767,0.6805,9.839357,3.900061,5.421687,0.655573,1.405622,8
7,SYF,35.400,N/A,5.71,9.236948,1.3000,15.060241,0.8853,13.453815,3.414872,3.614458,1.038622,4.417671,9
8,F,13.950,N/A,6.25,10.441767,1.2400,14.257028,0.3430,3.212851,3.956095,5.823293,1.758730,10.040161,9
9,PVH,76.810,N/A,5.88,9.638554,0.9541,10.040161,0.5740,7.028112,5.585405,10.843373,1.237095,6.827309,9


# Calculating Number of Shares to Buy

In [13]:
portifolio_size = 100000
position_size = float(portifolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row,'Number of Shares to Buy'] = round(position_size/rv_dataframe.loc[row,'Price'])

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,KSS,27.700,7,6.480000,11.044177,0.745000,8.032129,0.165100,0.803213,4.599538,7.831325,1.137719,5.823293,7
1,GM,38.951,5,6.820000,11.84739,0.864000,9.036145,0.383000,4.417671,1.978254,1.807229,1.610315,9.236948,7
2,AIG,63.991,3,3.700000,5.62249,1.200000,13.554217,0.808200,12.248996,2.196776,2.008032,0.785324,2.610442,7
3,C,46.040,4,6.200000,10.240964,0.504000,7.429719,0.964200,14.859438,3.754686,4.618474,1.020876,4.016064,8
4,BA,183.570,1,-12.920000,3.413655,-6.180000,6.024096,1.790000,31.425703,-28.203796,0.60241,-172.958889,0.200803,8
5,COF,94.860,2,4.850000,7.228916,0.715500,7.831325,1.031500,16.666667,3.197437,3.012048,1.077321,5.02008,8
6,UNM,40.950,5,6.960000,12.248996,0.972800,10.441767,0.680500,9.839357,3.900061,5.421687,0.655573,1.405622,8
7,SYF,35.400,6,5.710000,9.236948,1.300000,15.060241,0.885300,13.453815,3.414872,3.614458,1.038622,4.417671,9
8,F,13.950,14,6.250000,10.441767,1.240000,14.257028,0.343000,3.212851,3.956095,5.823293,1.758730,10.040161,9
9,PVH,76.810,3,5.880000,9.638554,0.954100,10.040161,0.574000,7.028112,5.585405,10.843373,1.237095,6.827309,9


# Formatting our excel output

In [14]:
writer = pd.ExcelWriter('value_strategy2.xlsx', engine = 'xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)


In [15]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [16]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [17]:
writer.save()